# **Method - 2**

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [2]:
project_path = "/content/drive/MyDrive/transformertranslator/"

In [3]:
# read phrases from english_telugu_data.txt file
english_sentances = []
telugu_sentances = []
with open(project_path+"english_telugu_data.txt", mode='rt', encoding='utf-8') as fp:
    for line in fp.readlines():
        eng_tel = line.split("++++$++++")
        english_sentances.append(eng_tel[0])
        telugu_sentances.append(eng_tel[1])

In [4]:
data = pd.DataFrame({"english_sentances":english_sentances,"telugu_sentances":telugu_sentances})

In [5]:
data.head()

,english_sentances,telugu_sentances
0,His legs are long.,అతని కాళ్ళు పొడవుగా ఉన్నాయి.\n
1,Who taught Tom how to speak French?,టామ్ ఫ్రెంచ్ మాట్లాడటం ఎలా నేర్పించారు?\n
2,I swim in the sea every day.,నేను ప్రతి రోజు సముద్రంలో ఈత కొడతాను.\n
3,Tom popped into the supermarket on his way hom...,టామ్ కొంచెం పాలు కొనడానికి ఇంటికి వెళ్ళేటప్పుడ...
4,Smoke filled the room.,పొగ గదిని నింపింది.\n


In [6]:
data.tail()

,english_sentances,telugu_sentances
155993,"The hand gesture, called ""Abhaya Mudra,"" repre...","""అభయ ముద్ర,"" అని పిలుస్తారు, బాధితులకు ఆత్మవిశ..."
155994,It symbolizes the act of listening deeply to t...,అది ప్రజల దుఃఖానికి ఆత్మగా వినియోగం చేస్తుంది ...
155995,It symbolizes the Bodhisattva's vow to listen ...,ఇది బోధిసత్త్వ ప్రతిజ్ఞను సూచిస్తుంది ప్రపంచాన...
155996,Avalokiteshvara's hand gestures convey various...,అవలోకితేశ్వర చేతుల చిహ్నాలు వివిధ అర్థాలను సూచ...
155997,Buddhist teachings inspire art by conveying pr...,బౌద్ధ ఉపదేశాలు ఆళ్లలో గంభీరమైన ఆధ్యాత్మిక అంశా...


In [7]:
data.shape

(155998, 2)

In [8]:
data.shape

(155998, 2)

In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [10]:
# clean english sentances
def clean_eng(text):
    # Lowercase all characters
    text = text.lower()
    # map contractions
    text = ' '.join([contraction_mapping[w] if w in contraction_mapping else w for w in text.split(" ")])
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove extra spaces
    text= text.strip()

    return text

In [11]:
# clean telugu sentances
def clean_tel(text):
    # Lowercase all characters
    text = text.lower()
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove Telugu numbers from text
    text = re.sub("[౦౧౨౩౪౫౬౭౮౯]", '', text)
    # Remove extra spaces
    text= text.strip()
    text = 'START_ '+ text + ' _END'
    return text

In [12]:
# clean text
data_df = data.copy()
data_df["english_sentances"] = data_df["english_sentances"] .apply(lambda x: clean_eng(x))
data_df["telugu_sentances"] = data_df["telugu_sentances"] .apply(lambda x: clean_tel(x))

In [13]:
# Vocabulary of English
all_eng_words=set()
for eng in data_df.english_sentances:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French
all_telugu_words=set()
for tel in data_df.telugu_sentances:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

In [14]:
# Max Length of source sequence
lenght_list=[]
for l in data_df.english_sentances:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

101

In [15]:
# Max Length of target sequence
lenght_list=[]
for l in data_df.telugu_sentances:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

30

In [16]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens

(14027, 38960)

In [17]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

38961

In [18]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [19]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [20]:
data_df = shuffle(data_df)
data_df.head(10)

,english_sentances,telugu_sentances
76414,she ignored him all day,START_ ఆమె రోజంతా అతన్ని పట్టించుకోలేదు _END
89802,i had to go back,START_ నేను తిరిగి వెళ్ళవలసి వచ్చింది _END
139684,tom turned to mary,START_ టామ్ మేరీ వైపు తిరిగింది _END
88529,the pain was intolerable,START_ నొప్పి భరించలేనిది _END
31043,i do not know how long it takes to get to bost...,START_ ఇక్కడి నుండి బోస్టన్‌కు వెళ్లడానికి ఎంత...
39289,the enemy gave in without further resistance,START_ మరింత ప్రతిఘటన లేకుండా శత్రువు ఇచ్చాడు ...
151642,who can tell me how a light bulb works,START_ లైట్ బల్బ్ ఎలా పనిచేస్తుందో నాకు ఎవరు చ...
15412,there was a traffic jam,START_ ట్రాఫిక్ జామ్ ఉంది _END
90098,i am warm,START_ నేను వెచ్చగా ఉన్నాను _END
66642,how long are you going to stay here,START_ మీరు ఎంతకాలం ఇక్కడే ఉండబోతున్నారు _END


In [21]:
# Train - Test Split
X, y = data_df.english_sentances, data_df.telugu_sentances
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((140398,), (15600,))

In [22]:
X_train.to_pickle(project_path+'X_train.pkl')
X_test.to_pickle(project_path+'X_test.pkl')

In [23]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [24]:
latent_dim = 50

In [25]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [26]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [28]:
batch_size = 128
epochs = 20
train_samples_steps = len(X_train) // batch_size
val_samples_steps = len(X_test) // batch_size

In [29]:
# generate train and test datra
train_gen = generate_batch(X_train, y_train, batch_size = batch_size)
test_gen = generate_batch(X_test, y_test, batch_size = batch_size)

In [30]:
# Defining a helper function to save the model after each epoch
# in which the loss decreases
filepath = project_path+'NMT_model_enc_dec.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# Defining a helper function to reduce the learning rate each time
# the learning plateaus
reduce_alpha = ReduceLROnPlateau(monitor ='val_loss', factor = 0.2,patience = 1, min_lr = 0.001)
# stop traning if there increase in loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
callbacks = [checkpoint, es, reduce_alpha]

In [31]:
from multiprocessing import cpu_count

# Get the number of CPU cores available
num_cores = cpu_count()
num_cores

2

In [ ]:
# # train the model
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_samples_steps,
                    epochs=epochs,
                    validation_data = test_gen,
                    validation_steps = val_samples_steps,callbacks = callbacks
                  )




<ipython-input-33-483c85a063ef>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = train_gen,


Epoch 1/20
1096/1096 [==============================] - ETA: 0s - loss: 6.6499 - acc: 0.2111
Epoch 1: val_loss improved from inf to 6.08324, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 857s 774ms/step - loss: 6.6499 - acc: 0.2111 - val_loss: 6.0832 - val_acc: 0.2269 - lr: 0.0010
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1096/1096 [==============================] - ETA: 0s - loss: 5.8934 - acc: 0.2334
Epoch 2: val_loss improved from 6.08324 to 5.71824, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 846s 772ms/step - loss: 5.8934 - acc: 0.2334 - val_loss: 5.7182 - val_acc: 0.2478 - lr: 0.0010
Epoch 3/20
1096/1096 [==============================] - ETA: 0s - loss: 5.5984 - acc: 0.2600
Epoch 3: val_loss improved from 5.71824 to 5.52233, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 846s 772ms/step - loss: 5.5984 - acc: 0.2600 - val_loss: 5.5223 - val_acc: 0.2729 - lr: 0.0010
Epoch 4/20
1096/1096 [==============================] - ETA: 0s - loss: 5.4040 - acc: 0.2944
Epoch 4: val_loss improved from 5.52233 to 5.32343, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 851s 

In [ ]:
# prompt: train additional 20 epochs from last check point

# Load the saved model
model.load_weights(filepath)

# Train the model for additional 20 epochs
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_samples_steps,
                    epochs=20,
                    validation_data = test_gen,
                    validation_steps = val_samples_steps,callbacks = callbacks
                  )


<ipython-input-35-044401d85895>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = train_gen,


Epoch 1/20
1096/1096 [==============================] - ETA: 0s - loss: 3.7359 - acc: 0.4760
Epoch 1: val_loss improved from inf to 3.74508, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1096/1096 [==============================] - 870s 794ms/step - loss: 3.7359 - acc: 0.4760 - val_loss: 3.7451 - val_acc: 0.4735 - lr: 0.0010
Epoch 2/20
1096/1096 [==============================] - ETA: 0s - loss: 3.6670 - acc: 0.4832
Epoch 2: val_loss improved from 3.74508 to 3.70516, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 866s 790ms/step - loss: 3.6670 - acc: 0.4832 - val_loss: 3.7052 - val_acc: 0.4774 - lr: 0.0010
Epoch 3/20
1096/1096 [==============================] - ETA: 0s - loss: 3.6013 - acc: 0.4902
Epoch 3: val_loss improved from 3.70516 to 3.66775, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 870s 794ms/step - loss: 3.6013 - acc: 0.4902 - val_loss: 3.6678 - val_acc: 0.4813 - lr: 0.0010
Epoch 4/20
1096/1096 [==============================] - ETA: 0s - loss: 3.5386 - acc: 0.4968
Epoch 4: val_loss improved from 3

In [ ]:
# Load the saved model
model.load_weights(filepath)

# Train the model for additional 20 epochs
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_samples_steps,
                    epochs=30,
                    validation_data = test_gen,
                    validation_steps = val_samples_steps,callbacks = callbacks
                  )

<ipython-input-33-8f1f96697528>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = train_gen,


Epoch 1/30
1096/1096 [==============================] - ETA: 0s - loss: 2.4369 - acc: 0.6161
Epoch 1: val_loss did not improve from 2.49460
1096/1096 [==============================] - 899s 820ms/step - loss: 2.4369 - acc: 0.6161 - val_loss: 2.5002 - val_acc: 0.6075 - lr: 0.0010
Epoch 2/30
1096/1096 [==============================] - ETA: 0s - loss: 2.3985 - acc: 0.6209
Epoch 2: val_loss did not improve from 2.49460
1096/1096 [==============================] - 899s 820ms/step - loss: 2.3985 - acc: 0.6209 - val_loss: 2.4948 - val_acc: 0.6083 - lr: 0.0010
Epoch 3/30
1096/1096 [==============================] - ETA: 0s - loss: 2.3628 - acc: 0.6249
Epoch 3: val_loss improved from 2.49460 to 2.48566, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5
1096/1096 [==============================] - 902s 823ms/step - loss: 2.3628 - acc: 0.6249 - val_loss: 2.4857 - val_acc: 0.6092 - lr: 0.0010
Epoch 4/30
1096/1096 [==============================] - ETA: 0s - loss: 2

In [32]:
# Load the saved model
model.load_weights(filepath)

# Train the model for additional 20 epochs
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_samples_steps,
                    epochs=20,
                    validation_data = test_gen,
                    validation_steps = val_samples_steps,callbacks = callbacks
                  )

<ipython-input-32-a0ac473dfc10>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = train_gen,


Epoch 1/20
1096/1096 [==============================] - ETA: 0s - loss: 1.7128 - acc: 0.7034
Epoch 1: val_loss improved from inf to 1.71083, saving model to /content/drive/MyDrive/transformertranslator/NMT_model_enc_dec.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1096/1096 [==============================] - 1074s 969ms/step - loss: 1.7128 - acc: 0.7034 - val_loss: 1.7108 - val_acc: 0.7016 - lr: 0.0010
Epoch 2/20
1096/1096 [==============================] - ETA: 0s - loss: 1.6869 - acc: 0.7075
Epoch 2: val_loss did not improve from 1.71083
1096/1096 [==============================] - 1054s 961ms/step - loss: 1.6869 - acc: 0.7075 - val_loss: 1.7312 - val_acc: 0.6979 - lr: 0.0010
Epoch 3/20
1096/1096 [==============================] - ETA: 0s - loss: 1.6652 - acc: 0.7104
Epoch 3: val_loss did not improve from 1.71083
1096/1096 [==============================] - 1046s 954ms/step - loss: 1.6652 - acc: 0.7104 - val_loss: 1.7460 - val_acc: 0.6945 - lr: 0.0010
Epoch 3: early stopping


# ------Model Training Completed-------

Inference Setup

In [33]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)


# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Decode sample sequeces

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Evaluation on Train Dataset

In [35]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
for k in range(10):
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_test[k:k+1].values[0])
    print('Actual Telugu Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Telugu Translation:', decoded_sentence[:-4])
    print("\n")

1/1 [==============================] - 0s 22ms/step
Input English sentence: we liked each other
Actual Telugu Translation:  మేము ఒకరినొకరు ఇష్టపడ్డాము 
Predicted Telugu Translation:  మేము ఒకరినొకరు బాగా కనుగొన్నాము 


1/1 [==============================] - 0s 19ms/step
Input English sentence: tom said he could not walk
Actual Telugu Translation:  టామ్ తాను నడవలేనని చెప్పాడు 
Predicted Telugu Translation:  టామ్ తాను డ్రైవ్ చేయకూడదని చెప్పాడు 


1/1 [==============================] - 0s 18ms/step
Input English sentence: i did not think i would be late
Actual Telugu Translation:  నేను ఆలస్యం అవుతాను అని అనుకోలేదు 
Predicted Telugu Translation:  నేను ఆలస్యం అవుతాను అని అనుకోలేదు 


1/1 [==============================] - 0s 24ms/step
Input English sentence: we had a bad day
Actual Telugu Translation:  మాకు చెడ్డ రోజు వచ్చింది 
Predicted Telugu Translation:  మాకు చెడ్డ రోజు వచ్చింది 


1/1 [==============================] - 0s 60ms/step
Input English sentence: i think i should let tom leave


In [36]:
# prompt: based on this code above, now take user english sentence and translate it to telugu

import numpy as np
def translate_user_input(input_sentence):
  # Preprocess the input sentence
  input_sentence = clean_eng(input_sentence)
  # Convert the input sentence to a tokenized sequence
  input_seq = np.zeros((1, max_length_src))
  for i, word in enumerate(input_sentence.split()):
    if word in input_token_index:
      input_seq[0, i] = input_token_index[word]
  # Translate the input sentence
  translated_sentence = decode_sequence(input_seq)
  # Print the translated sentence
  print('Translated Telugu sentence:', translated_sentence[:-4])

# Get user input sentence
input_sentence = input("Enter an English sentence: ")

# Translate the user input sentence
translate_user_input(input_sentence)


Enter an English sentence: WHO INVENTED THIS?
1/1 [==============================] - 0s 19ms/step
Translated Telugu sentence:  ఈ ఎవరు కనుగొన్నారు 
